# XAUUSD Prediction Bot Analysis
This notebook visualizes the data, training process, and **Future Predictions** for the Hybrid LSTM-ARIMA Model.

> **Make sure to run this in the virtual environment!**

In [ ]:

import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

# Add source path
sys.path.append(os.path.abspath(''))

# Configure GPU if available
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"[INFO] GPU Enabled: {len(gpus)} device(s)")
    except RuntimeError as e:
        print(e)

from src.data_loader import DataLoader
from src.hybrid_model import HybridModel
from src.utils import calculate_metrics, get_steps_for_days

ImportError: cannot import name 'get_steps_for_days' from 'src.utils' (d:\Code\XAUUSD_Pred_Bot\src\utils.py)

## 1. Configuration
**Change these values to customize your analysis.**

In [ ]:
# --- CONFIGURATION --- #
SYMBOL = 'XAUUSD'       # Asset Symbol (e.g., 'XAUUSD', 'EURUSD')
TIMEFRAME = '1h'        # Timeframe: '1h', '30m', '15m', '4h', '1d'
PERIOD = '5y'           # History period to fetch: '1y', '2y', '5y', 'max'
FUTURE_DAYS = 5         # Number of days to predict into the future
EPOCHS = 10             # Training epochs
# --------------------- #

## 2. Load Data

In [ ]:
print(f"Fetching {SYMBOL} data ({TIMEFRAME})...")
loader = DataLoader(symbol=SYMBOL, interval=TIMEFRAME, period=PERIOD, data_path=f'data/gold_{TIMEFRAME}.csv')

# Fetch data (Try MT5 first, fallback to yfinance)
df = loader.fetch_data(source='mt5')
print(f"Loaded {len(df)} rows")
df.tail()

## 3. Preprocess

In [ ]:
look_back = 60
X, y, scaled_data = loader.prepare_data_for_lstm(df, look_back=look_back)

# 80/20 Split for Validation
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(f"Train samples: {X_train.shape[0]}, Test samples: {X_test.shape[0]}")

## 4. Train Hybrid Model

In [ ]:
input_shape = (X.shape[1], X.shape[2])
model = HybridModel(input_shape)

print("Training...")
model.train(X_train, y_train, epochs=EPOCHS)

## 5. Evaluate (Test Set)

In [ ]:
final_preds, lstm_preds, arima_preds = model.predict(X_test)

metrics = calculate_metrics(y_test, final_preds)
print("Hybrid Metrics:", metrics)

plt.figure(figsize=(14, 7))
plt.plot(y_test, label="Actual", color='black')
plt.plot(final_preds, label="Hybrid Prediction", color='red', linestyle='-.')
plt.title(f"{SYMBOL} {TIMEFRAME} - Test Set Evaluation")
plt.legend()
plt.show()

## 6. Future Prediction

In [ ]:
if FUTURE_DAYS > 0:
    steps = get_steps_for_days(TIMEFRAME, FUTURE_DAYS)
    print(f"Predicting next {FUTURE_DAYS} days -> {steps} candles ({TIMEFRAME})")
    
    # Use the LAST available sequence to start prediction
    last_sequence = scaled_data[-look_back:].reshape(1, look_back, 1)
    
    future_final, future_lstm, future_arima = model.predict_future(last_sequence, steps)
    
    # Inverse Scale
    future_prices = loader.inverse_transform(future_final)
    
    # Plot
    plt.figure(figsize=(12, 6))
    plt.plot(future_prices, label=f'Future {FUTURE_DAYS} Days', color='green')
    plt.title(f"Future Price Forecast: {SYMBOL}")
    plt.xlabel("Steps Ahead")
    plt.ylabel("Price")
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()
    
    print("Next 5 Predicted Close Prices:")
    print(future_prices[:5].flatten())
    print(f"... Final Price at step {steps}: {future_prices[-1,0]:.2f}")